In [1]:
import pandas as pd
import os

os.listdir('../data')

['accident11.xlsx',
 'accident_new.csv',
 'airpollution36912.csv',
 'carspeed_201801.csv',
 'carspeed_201804.csv',
 'carspeed_201807.csv',
 'carspeed_201809.csv',
 'carspeed_201810.csv',
 'carspeed_201812.csv',
 'dust1.csv',
 'dust_final.csv',
 'final',
 'gugucon.csv',
 'holiday.csv',
 'intersection.csv',
 'road_info.csv',
 'weather_merged.csv']

In [69]:
carspeed1 = pd.read_csv('../data/carspeed_201801.csv')
carspeed4 = pd.read_csv('../data/carspeed_201804.csv')
carspeed7 = pd.read_csv('../data/carspeed_201807.csv')
carspeed9 = pd.read_csv('../data/carspeed_201809.csv')
carspeed10 = pd.read_csv('../data/carspeed_201810.csv')
carspeed12 = pd.read_csv('../data/carspeed_201812.csv')

print('shape 1 : ', carspeed1.shape)
print('shape 4 : ', carspeed4.shape)
print('shape 7 : ', carspeed7.shape)
print('shape 9 : ', carspeed9.shape)
print('shape 10 : ', carspeed10.shape)
print('shape 12 : ', carspeed12.shape)

shape 1 :  (147466, 22)
shape 4 :  (8940, 25)
shape 7 :  (146543, 25)
shape 9 :  (141827, 26)
shape 10 :  (9238, 26)
shape 12 :  (9238, 26)


In [70]:
carspeed7.columns

Index(['일자', '요일', '도로명', '링크아이디', '시점명', '종점명', '방향', '거리', '기능유형구분',
       '도심/외곽구분', '권역구분', '10시', '11시', '12시', '13시', '14시', '15시', '16시',
       '17시', '18시', '19시', '20시', '21시', '22시', '23시'],
      dtype='object')

# common link

In [71]:
common_link = set(carspeed1.링크아이디).intersection(set(carspeed4.링크아이디)).intersection(set(carspeed7.링크아이디)).intersection(set(carspeed9.링크아이디)).intersection(set(carspeed10.링크아이디)).intersection(set(carspeed12.링크아이디))
len(common_link)

298

In [72]:
carspeed1 = carspeed1[carspeed1.링크아이디.isin(common_link)]
carspeed4 = carspeed4[carspeed4.링크아이디.isin(common_link)]
carspeed7 = carspeed7[carspeed7.링크아이디.isin(common_link)]
carspeed9 = carspeed9[carspeed9.링크아이디.isin(common_link)]
carspeed10 = carspeed10[carspeed10.링크아이디.isin(common_link)]
carspeed12 = carspeed12[carspeed12.링크아이디.isin(common_link)]

print('shape 1 : ', carspeed1.shape)
print('shape 4 : ', carspeed4.shape)
print('shape 7 : ', carspeed7.shape)
print('shape 9 : ', carspeed9.shape)
print('shape 10 : ', carspeed10.shape)
print('shape 12 : ', carspeed12.shape)

shape 1 :  (9238, 22)
shape 4 :  (8940, 25)
shape 7 :  (9238, 25)
shape 9 :  (8931, 26)
shape 10 :  (9238, 26)
shape 12 :  (9238, 26)


In [73]:
set(carspeed12.columns) - set(carspeed9.columns)

set()

In [74]:
carspeed9[carspeed9.차선수.isnull()].링크아이디.value_counts()

1230001500    30
1230001600    30
Name: 링크아이디, dtype: int64

In [75]:
carspeed9.loc[carspeed9.링크아이디 == 1230001500, '차선수'] = 0
carspeed9.loc[carspeed9.링크아이디 == 1230001600, '차선수'] = 0

# road info

In [76]:
road_features = carspeed12.groupby('링크아이디')['권역구분', '기능유형구분','도심/외곽구분','차선수'].first()
road_features = road_features.reset_index()
road_features.head(3)

,링크아이디,권역구분,기능유형구분,도심/외곽구분,차선수
0,1210002700,강남구,주간선도로,외곽,4
1,1210006400,강남구,도시고속도로,외곽,5
2,1210007000,강남구,주간선도로,외곽,3


In [77]:
carspeed1 = pd.merge(carspeed1, road_features[['링크아이디', '차선수','기능유형구분', '권역구분', '도심/외곽구분']], how = 'left')
carspeed4 = pd.merge(carspeed4, road_features[['링크아이디', '차선수']], how = 'left')
carspeed7 = pd.merge(carspeed7, road_features[['링크아이디', '차선수']], how = 'left')

#column 순서
carspeed1 = carspeed1.loc[:,carspeed12.columns]
carspeed4 = carspeed4.loc[:,carspeed12.columns]
carspeed7 = carspeed7.loc[:,carspeed12.columns]
carspeed9 = carspeed9.loc[:,carspeed12.columns]
carspeed10 = carspeed10.loc[:,carspeed12.columns]

print('shape 1 : ', carspeed1.shape)
print('shape 4 : ', carspeed4.shape)
print('shape 7 : ', carspeed7.shape)
print('shape 9 : ', carspeed9.shape)
print('shape 10 : ', carspeed10.shape)
print('shape 12 : ', carspeed12.shape)

shape 1 :  (9238, 26)
shape 4 :  (8940, 26)
shape 7 :  (9238, 26)
shape 9 :  (8931, 26)
shape 10 :  (9238, 26)
shape 12 :  (9238, 26)


# data 합치기

In [78]:
carspeed = pd.concat([carspeed1, carspeed4, carspeed7, carspeed9, carspeed10, carspeed12])
del carspeed1, carspeed4, carspeed7, carspeed9, carspeed10, carspeed12 #메모리 관리를 위해 각 월 별 데이터는 삭제

In [79]:
carspeed.isnull().sum()

일자           0
요일           0
도로명          0
링크아이디        0
시점명          0
종점명          0
방향           0
거리           0
차선수          0
기능유형구분       0
도심/외곽구분      0
권역구분         0
10시        303
11시        603
12시         24
13시          3
14시          1
15시          1
16시          1
17시        299
18시          4
19시         21
20시          7
21시          4
22시          4
23시          3
dtype: int64

# 중복값 삭제

In [80]:
carspeed[['링크아이디','10시', '11시', '12시', '13시', '14시', '15시', '16시','17시', '18시', '19시', '20시', '21시', '22시', '23시']].duplicated().sum()

705

In [81]:
col_dup = ['링크아이디','10시', '11시', '12시', '13시', '14시', '15시', '16시','17시', '18시', '19시',
                            '20시', '21시', '22시', '23시']

carspeed = carspeed.drop_duplicates(col_dup, keep = False)
carspeed.shape

(54070, 26)

# holiday

In [82]:
holiday = pd.read_csv('../data/holiday.csv',sep = '\t')
holiday.rename(columns = {'date' : '일자'}, inplace = True)
holiday.head(3)

,일자,holiday,special,vacation
0,20180101,1,0,0
1,20180102,0,0,0
2,20180103,0,0,0


In [83]:
carspeed = pd.merge(carspeed, holiday, on = '일자')
carspeed.shape

(54070, 29)

# melt

In [84]:
col = list(carspeed.columns[:12]) + ['holiday','special', 'vacation'] + list(carspeed.columns[12:26])
carspeed = carspeed[col] # y를 맨 뒤로 보내기 위한 작업
carspeed.head(3)

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20180101,월,강남대로,1220021100,우성아파트,강남역,상행,586,5.0,주간선도로,...,24.54,24.78,25.33,23.92,23.55,23.56,25.49,23.63,25.61,22.44
1,20180101,월,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,5.0,주간선도로,...,30.99,28.08,27.79,26.96,28.18,27.47,27.71,27.42,26.72,28.18
2,20180101,월,개포로,1220020500,대청초교,대청역,상행,862,2.0,기타도로,...,22.22,28.02,23.21,20.56,20.66,24.62,23.78,28.08,32.25,28.37


In [85]:
col = carspeed.columns[:15]
carspeed = pd.melt(carspeed, id_vars= col)
print(carspeed.shape)
carspeed.head(3)

(756980, 17)


,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,권역구분,holiday,special,vacation,variable,value
0,20180101,월,강남대로,1220021100,우성아파트,강남역,상행,586,5.0,주간선도로,외곽,강남구,1,0,0,10시,26.26
1,20180101,월,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,5.0,주간선도로,외곽,강남구,1,0,0,10시,34.66
2,20180101,월,개포로,1220020500,대청초교,대청역,상행,862,2.0,기타도로,외곽,강남구,1,0,0,10시,24.66


In [86]:
carspeed.columns = ['date','days','road_name','link_id','start_point','end_point','direction','road_length','road_num',
                    'road_type','urban_suburb','district','holiday', 'special', 'vacation','time','y']
carspeed.time = carspeed.time.apply(lambda x : int(x[:2]))

In [87]:
carspeed.shape

(756980, 17)

# intersection

In [88]:
intersection = pd.read_csv('../data/intersection.csv')
intersection = intersection.melt(id_vars = '도로명')
intersection.columns = ['road_name','direction','inter_num']
intersection.head(3)

,road_name,direction,inter_num
0,4.19로,상행,2.0
1,가락로,상행,4.0
2,가로공원로,상행,4.0


In [89]:
carspeed = pd.merge(carspeed, intersection, how = 'left', left_on = ['road_name','direction'], right_on = ['road_name','direction'])
carspeed.shape

(756980, 18)

# accident_new

In [90]:
accident_new = pd.read_csv('../data/accident_new.csv')
accident_new.columns = ['road_name','d1','d2','d3','d4','victims','severity','accident_count']
accident_new.head(3)

,road_name,d1,d2,d3,d4,victims,severity,accident_count
0,4.19로,0,3,15,1,19,55,13
1,63로,0,0,3,1,4,10,3
2,가락로,0,20,50,15,85,225,70


In [91]:
carspeed = pd.merge(carspeed, accident_new, how = 'left', left_on = ['road_name'], right_on = ['road_name'])
carspeed.shape

(756980, 25)

# weather

In [92]:
weather = pd.read_csv('../data/weather_merged.csv')
weather = weather[weather.time !=0]
weather = weather.drop(['지점','일시', 'id'],axis = 1)
weather.rename(columns = {'일자' : 'date'}, inplace = True)
weather['date'] = weather['date'].str.replace('-','')
weather.head()

,temperature,wind,rain,date,district,time,snow
1,-0.9,0.9,0.0,20180101,강남구,1,0.0
2,-1.0,1.1,0.0,20180101,강남구,2,0.0
3,-1.1,1.5,0.0,20180101,강남구,3,0.0
4,-1.1,1.8,0.0,20180101,강남구,4,0.0
5,-1.4,1.3,0.0,20180101,강남구,5,0.0


In [93]:
#type 맞춰주기
carspeed.date = carspeed.date.astype(str)
carspeed.shape

(756980, 25)

In [94]:
carspeed = pd.merge(carspeed, weather,left_on=['date','time','district'], right_on=['date','time','district'],how = 'inner')
carspeed.shape

(748706, 29)

# 도로정보

In [95]:
road_info = pd.read_csv('../data/road_info.csv')
road_info.head()

,link_id,cctv,kids,speed_limit,entrance,signal
0,1180021600,0,1,30,21,30
1,1180021200,0,1,30,21,30
2,1180019600,0,0,30,21,30
3,1180018800,0,0,30,21,30
4,1180018900,0,0,30,21,30


In [96]:
road_info.isnull().sum()

link_id        0
cctv           0
kids           0
speed_limit    0
entrance       0
signal         0
dtype: int64

In [97]:
carspeed = pd.merge(carspeed, road_info, on = 'link_id')
carspeed.shape

(748706, 34)

In [98]:
carspeed.isnull().sum()

date                0
days                0
road_name           0
link_id             0
start_point         0
end_point           0
direction           0
road_length         0
road_num            0
road_type           0
urban_suburb        0
district            0
holiday             0
special             0
vacation            0
time                0
y                 970
inter_num           0
d1                  0
d2                  0
d3                  0
d4                  0
victims             0
severity            0
accident_count      0
temperature         0
wind                0
rain                0
snow                0
cctv                0
kids                0
speed_limit         0
entrance            0
signal              0
dtype: int64

# 미세먼지

In [7]:
#dt = pd.read_csv('../data/dt_total.csv')
dust = pd.read_csv('../data/dust_final.csv')
dust['date'] = dust.date.str.replace('-','').astype(int)
dust.date = dust.date.apply(str)
dust.head(3)

,date,time,측정소명,SO2,CO,O3,NO2,PM10,PM2.5
0,20180101,1,중구,0.004,0.5,0.020,0.020,34.0,19.0
1,20180101,2,중구,0.004,0.4,0.024,0.016,27.0,14.0
2,20180101,3,중구,0.004,0.4,0.018,0.022,26.0,14.0


In [8]:
dust2 = pd.read_csv('../data/airpollution36912.csv')
dust2.head(3)

,date,time,측정소명,SO2,CO,O3,NO2,PM10,PM2.5
0,20180301,1,중구,0.003,0.6,0.023,0.029,7.0,4.0
1,20180301,2,중구,0.003,0.6,0.015,0.036,6.0,4.0
2,20180301,3,중구,0.003,0.5,0.026,0.024,10.0,7.0


In [9]:
dust = pd.concat([dust, dust2])
dust.isnull().sum()

date        0
time        0
측정소명        0
SO2      1923
CO       2219
O3       2127
NO2      1657
PM10     1960
PM2.5    2150
dtype: int64

In [10]:
def NA_mean(air, col, date, time) :
    return air[(air['date'] == date) & (air['time'] == time)][col].mean()

In [11]:
cols = ['SO2','CO','O3','NO2','PM10','PM2.5']
for col in cols :
    frame = dust[dust[col].isnull()]
    mean_value = frame.apply(lambda x : NA_mean(dust, col, x['date'],x['time']), axis =1)
    dust.loc[mean_value.index, col] = mean_value

In [12]:
dust.columns = ['date', 'time', 'district', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM2.5']

In [13]:
dust.isnull().sum()

date        0
time        0
district    0
SO2         0
CO          0
O3          0
NO2         0
PM10        0
PM2.5       0
dtype: int64

In [14]:
dust.shape

(147000, 9)

In [15]:
car_sm.shape

(747736, 34)

In [16]:
dust.date.apply(lambda x : str(x)[-4:-2]).value_counts()

01    18600
12    18600
07    18600
10    18600
03    18600
06    18000
04    18000
09    18000
Name: date, dtype: int64

In [17]:
car_sm.date.apply(lambda x : str(x)[-4:-2]).value_counts()

01    127508
10    127382
07    127344
09    123534
12    122758
04    119210
Name: date, dtype: int64

In [22]:
dust.date = dust.date.apply(int)

In [26]:
car_sm = pd.merge(car_sm, dust, on = ['date','time', 'district'])
car_sm.shape

(747736, 40)

In [28]:
#car_sm.to_csv('../data/final/car_sm.csv', index = False)

In [27]:
car_sm.isnull().sum()

date              0
days              0
road_name         0
link_id           0
start_point       0
end_point         0
direction         0
road_length       0
road_num          0
road_type         0
urban_suburb      0
district          0
holiday           0
special           0
vacation          0
time              0
y                 0
inter_num         0
d1                0
d2                0
d3                0
d4                0
victims           0
severity          0
accident_count    0
temperature       0
wind              0
rain              0
snow              0
cctv              0
kids              0
speed_limit       0
entrance          0
signal            0
SO2               0
CO                0
O3                0
NO2               0
PM10              0
PM2.5             0
dtype: int64

# final drop

In [126]:
carspeed.dropna(inplace = True)
carspeed.shape

(747736, 34)

In [127]:
carspeed.to_csv('../data/final/car_sm.csv', index = False)

# final merge

In [29]:
car_sm = pd.read_csv('../data/final/car_sm.csv')
car_ej = pd.read_csv('../data/final/car_ej.csv')

print(car_sm.shape)
print(car_ej.shape)

(747736, 40)
(741882, 40)


In [30]:
car_ej.head(3)

,date,days,road_name,link_id,start_point,end_point,direction,road_length,road_num,road_type,...,special,vacation,inter_num,d1,d2,d3,d4,victims,severity,accident_count
0,20180201,목,강남대로,1220021100,우성아파트,강남역,상행,586,5.0,주간선도로,...,0,0,15.0,3,69,338,28,438,1285,280
1,20180201,목,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,5.0,주간선도로,...,0,0,15.0,3,69,338,28,438,1285,280
2,20180201,목,개포로,1220020500,대청초교,대청역,상행,862,2.0,기타도로,...,0,0,8.0,0,21,31,5,57,161,39


In [31]:
car_sm.head(3)

,date,days,road_name,link_id,start_point,end_point,direction,road_length,road_num,road_type,...,kids,speed_limit,entrance,signal,SO2,CO,O3,NO2,PM10,PM2.5
0,20180101,월,강남대로,1220021100,우성아파트,강남역,상행,586,5.0,주간선도로,...,0,60,82,54,0.005,0.8,0.003,0.042,33.0,25.0
1,20180101,월,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,5.0,주간선도로,...,0,60,82,54,0.005,0.8,0.003,0.042,33.0,25.0
2,20180101,월,개포로,1220020500,대청초교,대청역,상행,862,2.0,기타도로,...,0,30,19,53,0.005,0.8,0.003,0.042,33.0,25.0


In [39]:
car_ej.rename(columns = {'value' : 'y'}, inplace = True)

In [40]:
car_ej = car_ej[car_sm.columns]

In [42]:
real_final = pd.concat([car_sm, car_ej])
real_final.shape

(1489618, 40)

In [66]:
real_final.to_csv('../data/final/real_final.csv', index = False, encoding = 'utf-8')

# train, test

In [53]:
real_final['month'] = real_final.date.apply(lambda x : int(str(x)[-4:-2]))
real_final['weekend'] = real_final.days.apply(lambda x : 1 if (x == '토') | (x == '일') else 0)

In [55]:
real_final.head(3)

,date,days,road_name,link_id,start_point,end_point,direction,road_length,road_num,road_type,...,entrance,signal,SO2,CO,O3,NO2,PM10,PM2.5,month,weekend
0,20180101,월,강남대로,1220021100,우성아파트,강남역,상행,586,5.0,주간선도로,...,82.0,54.0,0.005,0.8,0.003,0.042,33.0,25.0,1,0
1,20180101,월,강남대로,1220016200,우성아파트,뱅뱅사거리,하행,414,5.0,주간선도로,...,82.0,54.0,0.005,0.8,0.003,0.042,33.0,25.0,1,0
2,20180101,월,개포로,1220020500,대청초교,대청역,상행,862,2.0,기타도로,...,19.0,53.0,0.005,0.8,0.003,0.042,33.0,25.0,1,0


In [56]:
real_final.columns

Index(['date', 'days', 'road_name', 'link_id', 'start_point', 'end_point',
       'direction', 'road_length', 'road_num', 'road_type', 'urban_suburb',
       'district', 'holiday', 'special', 'vacation', 'time', 'y', 'inter_num',
       'd1', 'd2', 'd3', 'd4', 'victims', 'severity', 'accident_count',
       'temperature', 'wind', 'rain', 'snow', 'cctv', 'kids', 'speed_limit',
       'entrance', 'signal', 'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM2.5',
       'month', 'weekend'],
      dtype='object')

In [57]:
X = real_final.drop('y', axis = 1)
y = real_final['y']

In [58]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train ,y_test = train_test_split(X,y, random_state = 333, test_size = 0.3)

In [65]:
train = pd.concat([X_train, y_train], axis = 1)
test = pd.concat([X_test, y_test], axis = 1)

train.to_csv('../data/final/train.csv', index = False, encoding = 'utf-8')
test.to_csv('../data/final/test.csv', index = False, encoding = 'utf-8')